#  DCGAN Improvement 3


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os,cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pylab import rcParams
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import mean
from numpy.random import randn
from numpy.random import randint
from numpy.random import choice
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.optimizers import RMSprop
from keras.optimizers import Adam
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.models import Model
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import Concatenate
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import tensor_shape
from tensorflow.python.eager import def_function
from tensorflow.python.keras import initializers
from keras.initializers import RandomNormal
from tensorflow.python.keras import backend as K
from keras.utils import np_utils
from keras.utils import plot_model
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator
from numpy import asarray
import pickle

Using TensorFlow backend.


## Data Preprocessing 

In [2]:
#get the list of all files in the specified directory
path = 'CK PLUS/CK+48'
labels_list = os.listdir(path)
labels_list

images_data_list = []

for dataset in labels_list:
    imgs_per_label_list = os.listdir(path+'/'+ dataset)
    for image in imgs_per_label_list:
        #load the image in grayscale mode
        input_image = cv2.imread(path + '/'+ dataset + '/'+ image,0)
        #change the width and height of the original image to 48*48
        image_resized = cv2.resize(input_image,(48,48))
        #append the resized image to the images_data_list
        images_data_list.append(image_resized)

#convert data to numpy array
images_data = np.array(images_data_list)
images_data.shape

number_of_classes = 7
number_of_samples = images_data.shape[0]

#create an 1D array of zeros with 981 elements.
labels = np.zeros(number_of_samples,dtype='int64')

labels[0:206]=0 #207
labels[207:260]=1 #54
labels[261:335]=2 #75
labels[336:584]=3 #249
labels[585:668]=4 #84
labels[669:803]=5 #135
labels[804:980]=6 #177

#labels array
y = np.array(labels)
X = images_data.reshape(images_data.shape[0],48,48,1) 

#Shuffle the dataset
X,y = shuffle(X,y, random_state=42)

#Convert data into a float32 array 
X = X.astype('float32')
# scale from [0,255] to [-1,1]
X = (X - 127.5) / 127.5

df = pd.read_csv('icml_face_data.csv')
df_split = np.array_split(df, 3)
df = df_split[0]

df['emotion']=df['emotion'].replace({0:8,1:9,2:10,3:11,4:12,5:13,6:14}).replace({8:5,9:6,10:2,11:0,12:4,13:3,14:7})
df['emotion'].unique()

def preprocessing_data(df):  
    img_array = np.zeros(shape=(len(df), 48, 48))
    img_label = np.array(list(map(int, df['emotion']))) #map() function is used to iterate over an array
    
    for i, row in enumerate(df.index):
        img = np.fromstring(df.loc[row,' pixels'], dtype=int, sep=' ')
        img = np.reshape(img, (48, 48))
        img_array[i] = img
        
    return img_array, img_label

X_img, y_labels = preprocessing_data(df)

#Reshape images into (48,48,1) shape
X_img = X_img.reshape((X_img.shape[0], 48, 48, 1))
#Convert data into a float32 array 
X_img = X_img.astype('float32')

# scale from [0,255] to [-1,1]
X_img = (X_img - 127.5) / 127.5

X = np.concatenate((X, X_img))
y = np.concatenate((y, y_labels))

#Shuffle the dataset
X,y = shuffle(X,y, random_state=42)

print(X.shape)
print(y.shape)

(12944, 48, 48, 1)
(12944,)


### Mini batch Discrimination


In [3]:
class MinibatchDiscrimination(layers.Layer):

    def __init__(self, num_kernel, dim_kernel,kernel_initializer='glorot_uniform', **kwargs):
        self.num_kernel = num_kernel
        self.dim_kernel = dim_kernel
        self.kernel_initializer = kernel_initializer
        super(MinibatchDiscrimination, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.num_kernel*self.dim_kernel),
                                      initializer=self.kernel_initializer,
                                      trainable=True)
        super(MinibatchDiscrimination, self).build(input_shape)  

    def call(self, x):
        activation = tf.matmul(x, self.kernel)
        activation = tf.reshape(activation, shape=(-1, self.num_kernel, self.dim_kernel))
        tmp1 = tf.expand_dims(activation, 3)
        tmp2 = tf.transpose(activation, perm=[1, 2, 0])
        tmp2 = tf.expand_dims(tmp2, 0)
        
        diff = tmp1 - tmp2
        
        l1 = tf.reduce_sum(tf.math.abs(diff), axis=2)
        features = tf.reduce_sum(tf.math.exp(-l1), axis=2)
        return tf.concat([x, features], axis=1)        
        
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1] + self.num_kernel)

### Standalone Discriminator

In [4]:
def define_discriminator():
    # Gaussian Weight Initialization
    init = RandomNormal(mean=0.0, stddev=0.02)
    model = Sequential()
    # downsample
    model.add(Conv2D(128, 3, input_shape=(48,48,1)))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128,4,strides=2,padding='same',kernel_initializer=init))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128,4,strides=2,padding='same',kernel_initializer=init))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(64,4,strides=2,padding='same',kernel_initializer=init))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(alpha=0.2))
    #flatten
    model.add(Flatten())
    model.add(Dense(64))
    model.add(LeakyReLU(alpha=0.2))
    #Mini batch Discrimination
    model.add(MinibatchDiscrimination(num_kernel=100, dim_kernel=5, name="mbd"))
    #classifier
    model.add(Dense(64))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5, clipvalue=1.0)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model 
    

In [5]:
# model summary
discriminator_improv_3 = define_discriminator()
discriminator_improv_3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 128)       1280      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 46, 46, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 128)       262272    
_________________________________________________________________
batch_normalization_1 (Batch (None, 23, 23, 128)       512       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 23, 23, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 128)       262272    
_________________________________________________________________
batch_normalization_2 (Batch (None, 12, 12, 128)      

2022-09-20 11:53:51.330573: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-20 11:53:51.331187: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


###  Keep all the rest and train


In [ ]:
def define_generator():
    # Gaussian Weight Initialization
    init = RandomNormal(mean=0.0, stddev=0.02)
    model = Sequential()
    # Transforming the input into a 6*6 128-channel feature map 
    model.add(Dense(128*6*6,kernel_initializer=init,input_dim=100))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((6, 6, 128)))
   # upsample to 12x12
    model.add(Conv2DTranspose(128,4, strides=2, padding='same', kernel_initializer=init))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 24*24
    model.add(Conv2DTranspose(128,4,strides=2, padding='same', kernel_initializer=init))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 48*48
    model.add(Conv2DTranspose(64,4,strides=2, padding='same', kernel_initializer=init))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(alpha=0.2))
    # Generator model, which maps the input of shape (dim_latent_space) into an image of shape (48,48,1)
    model.add(Conv2D(1, 7, activation='tanh', padding='same', kernel_initializer=init))
    return model

def define_gan(generator, discriminator):
    # Sets the discriminator weigths to non-trainable (this will only apply to the gan model)
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    opt =  Adam(lr=0.0002, beta_1=0.5, clipvalue=1.0)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

# select real samples
def real_samples(dataset,batch_size):
    # choose random images
    i = randint(0, dataset.shape[0],batch_size)
    # select images
    X = dataset[i]
    # generate class labels
    y = ones((batch_size, 1))
    y = y*0.9
    return X, y

def fake_samples(generator, dim_latent_space, num_points):
    # generate points in latent space
    X_input = latent_points(dim_latent_space, num_points)
    # predict outputs
    X = generator.predict(X_input)
    # create class labels
    y = zeros((num_points, 1))
    # number of labels to flip (5%)
    num_to_flip = int(0.05 * y.shape[0])
    # choose labels to flip
    flip_xi = choice([i for i in range(y.shape[0])], size=num_to_flip)
    # invert the labels in place
    y[flip_xi] = 1 - y[flip_xi] 
    return X, y

def latent_points(dim_latent_space,num_points):
    # generate points in the latent space
    X_input = randn(dim_latent_space * num_points) #Return samples from the “standard normal” distribution
    # reshape into a batch of inputs for the network
    X_input = X_input.reshape(num_points, dim_latent_space)
    return X_input

# create a line plot of loss for the gan and save to file
def plot_history(loss1_disc,loss2_disc, g_loss_hist):
    # plot history
    plt.plot(loss1_disc, label='discriminator_real_loss')
    plt.plot(loss2_disc, label='discriminator_fake_loss')
    plt.plot(g_loss_hist, label='generator_loss')
    plt.legend()
    plt.close()

def train(generator_model, discriminator_model, gan_model, dataset, dim_latent_space, num_epochs=9, batch_size=128):
    batch_per_epoch = int(dataset.shape[0] / batch_size)
    half_batch = int(batch_size / 2)
    # lists for keeping track of loss
    loss1_disc,loss2_disc, g_loss_hist = list(), list(), list()
    # manually enumerate epochs
    for i in range(num_epochs):
        loss1, loss2, g_loss = list(), list(),list()
        # enumerate batches over the training set
        for j in range(batch_per_epoch):
            # get randomly selected "real" samples
            X_real, y_real = real_samples(dataset, half_batch)
            # update discriminator model weights, One sided label smoothing
            discriminator_loss1, _ = discriminator_model.train_on_batch(X_real, y_real) #train_on_batch allows you to expressly update weights based on a collection of samples you provide, without regard to any fixed batch size.
            loss1.append(discriminator_loss1)
            # generate "fake" samples
            X_fake, y_fake = fake_samples(generator_model, dim_latent_space, half_batch)
            # update discriminator model weights
            discriminator_loss2, _ = discriminator_model.train_on_batch(X_fake, y_fake)
            loss2.append(discriminator_loss2)
            # Produce points in latent space to use as the generator's input
            X_gan = latent_points(dim_latent_space,batch_size)
            # create inverted labels for the fake samples
            y_gan = ones((batch_size, 1))
            # update the generator via the discriminator's error
            generator_loss = gan_model.train_on_batch(X_gan, y_gan)
            g_loss.append(generator_loss)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                (i+1, j+1, batch_per_epoch, discriminator_loss1, discriminator_loss2, generator_loss))
        # store losses per epoch 
        loss1_disc.append(mean(loss1))
        loss2_disc.append(mean(loss2))
        g_loss_hist.append(mean(g_loss))
    # save generator model to disk
    filename = 'generator_improv_3.sav'
    pickle.dump(generator_model, open(filename, 'wb'))
    # line plots of loss
    plot_history(loss1_disc,loss2_disc, g_loss_hist)

# size of the latent space
dim_latent_space = 100
# create the discriminator
discriminator_model= define_discriminator()
# create the generator
generator_model = define_generator()
# create the gan
gan_model = define_gan(generator_model, discriminator_model)
# load image data
dataset = X
# train model
train(generator_model,discriminator_model, gan_model, dataset, dim_latent_space)


### Loading the generator model and generating images

In [ ]:
# create and save a plot of generated images (reversed grayscale)
def g_img_plot(samples, n):
    # plot images
    for i in range(n * n):
        # define subplot
        plt.subplot(n, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.imshow(samples[i, :, :, 0], cmap='gray_r')
    plt.show()

# load the model from disk
model = pickle.load(open('generator_improv_3.sav', 'rb'))
# generate images
latent_points = latent_points(100, 100)
# generate images
output = model.predict(latent_points)
# plot generated images
g_img_plot(output,4)